## From Raw SASE → 220-DBM → 40 eV Spectral Filter

This notebook:
1. Builds a raw SASE seed on a small `(t,x,y)` grid  
2. Passes it through the 220-Laue double-bounce monochromator (DBM)  
3. Applies a 40 eV FWHM Gaussian spectral filter  
4. Plots the time-domain intensity at each stage 

In [ ]:
# %% [code]
import os, sys
# point Python at the directory containing tools.py, new_wave.py, etc.
sys.path.insert(0, '/mnt/data')

import numpy as np
import matplotlib.pyplot as plt
from types import SimpleNamespace

# dynamically load our seed‐builder routines
import importlib.util
spec = importlib.util.spec_from_file_location("tools", "/mnt/data/tools.py")
tools = importlib.util.module_from_spec(spec)
spec.loader.exec_module(tools)

Ocelot_SASE_seed_pstxy         = tools.Ocelot_SASE_seed_pstxy
Ocelot_SASE_seed_220_dbm_pstxy = tools.Ocelot_SASE_seed_220_dbm_pstxy

# %% [code]
# 1) Configuration object X
X = SimpleNamespace()

# ––– small time grid for demo
X.tgrid = 512
X.tmax  = 2e-12       # 2 ps window
X.dt    = X.tmax/X.tgrid
X.t     = np.linspace(0, X.tmax, X.tgrid)

# ––– small transverse grid
X.xgrid = 16
X.ygrid = 16
X.xmax  = 50e-6       # ±50 μm
X.ymax  = 50e-6
X.dx    = 2*X.xmax/X.xgrid
X.dy    = 2*X.ymax/X.ygrid

# ––– seed & photon params
X.E_seed_uJ          = 5.0
X.seed_FEL_bandwidth = 1e-3
X.seed_delay         = 0.0

X.config = {
  'seed_width_FWHM_x':   30e-6,
  'seed_width_FWHM_y':   30e-6,
  'seed_duration_FWHM_t': 50e-15,
}

# Cu Kα₁ photon
h = 6.62607015e-34; c = 2.99792458e8; e = 1.602176634e-19
X.lambdaKalpha1N = 1.5418e-10
X.hwKalpha1N     = (h*c/X.lambdaKalpha1N)/(h/(2*np.pi))
X.Gamma_sp_fsm1N = 3.0e8

# %% [code]
# 2) Generate raw & DBM‐filtered seeds
Ω_raw = Ocelot_SASE_seed_pstxy(X)
Ω_dbm = Ocelot_SASE_seed_220_dbm_pstxy(X)

# %% [code]
# 3) Extract σ→π at beam centre
xc = X.xgrid//2
yc = X.ygrid//2
E_raw = Ω_raw[0,1,:,xc,yc]
E_dbm = Ω_dbm[0,1,:,xc,yc]

# time axis in fs
tfs = X.t * 1e15

# %% [code]
# 4) Plot raw vs DBM (normalized)
I_raw = np.abs(E_raw)**2
I_dbm = np.abs(E_dbm)**2

plt.figure(figsize=(8,3))
plt.plot(tfs, I_raw/np.max(I_raw), label='Raw SASE (norm)')
plt.plot(tfs, I_dbm/np.max(I_dbm), label='After 220-DBM (norm)')
plt.xlabel('Time (fs)')
plt.ylabel('Normalized Intensity')
plt.title('Raw vs. DBM-Filtered Pulse')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# %% [code]
# 5) Spectral filter: 40 eV FWHM Gaussian around 8047.98 eV
E0_eV   = 8047.98
FWHM_eV =   40.0
sigma_E = (FWHM_eV*e)/(2*np.sqrt(2*np.log(2)))

Nt     = len(E_dbm)
E_dbm_f = np.fft.fft(E_dbm)
freq    = np.fft.fftfreq(Nt, d=X.dt)
omega   = 2*np.pi*freq
hbar    = h/(2*np.pi)

S_omega = np.exp(-0.5*((hbar*omega - E0_eV*e)/sigma_E)**2)
E_out_f = E_dbm_f * S_omega
E_out_t = np.fft.ifft(E_out_f)

# %% [code]
# 6) Plot DBM vs. spectrally filtered (normalized)
I_out = np.abs(E_out_t)**2

plt.figure(figsize=(8,3))
plt.plot(tfs, I_dbm/np.max(I_dbm), label='220-DBM (norm)')
plt.plot(tfs, I_out/np.max(I_out), label='After 40 eV Filter (norm)')
plt.xlabel('Time (fs)')
plt.ylabel('Normalized Intensity')
plt.title('Effect of 40 eV Spectral Filtering')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


AttributeError: 'NoneType' object has no attribute 'loader'